In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)



inp = Input(shape=(320, 240, 1))

n_filters = 512
max_p = inp

for _ in range(3):
    conv = Conv2D(filters=n_filters, kernel_size=5, activation='relu')(max_p)
    max_p = MaxPooling2D(pool_size=(2, 2))(conv)
    
    n_filters //= 2

conv = Conv2D(filters=n_filters, kernel_size=3, activation='relu')(max_p)
max_p = MaxPooling2D(pool_size=(2, 2))(conv)

flatten = Flatten()(max_p)
dense = Dense(128, activation='relu')(flatten)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)




model = Model(
    inputs=inp,
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        bicep_out,
        thigh_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        wrist_out,
        shoulder_to_shoulder_out,
    ],
    name='conv_bodies'
)


###TRAINING VARIABLES

sample_count_surreact = 23
sample_count_bodym = 12
epoch_count = 100
batch_size = 5
validation_count = 20
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
model_name="BoMN"

folder_path = f"{model_name} - SR{sample_count_surreact+sample_count_bodym}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")
# 
# model.summary()
# 
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


2024-05-18 17:37:06.776117: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 17:37:07.871762: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class Database_Loader(keras.utils.Sequence):
    
    def __init__(self, image_location, data_location, samples, batch_size, dataset, shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", side=False, height = False):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.side = side
        self.height = height

        super().__init__(workers=2, use_multiprocessing=True)
        self.IDs = []
        self._load_data(samples, data_location, image_location, dataset)
        self.on_epoch_end()      


    def __len__(self):
        return int(np.floor(len(self.IDs) / self.batch_size))
    
    
    def _load_data(self, samples, data_locations, image_locations, datasets):
        self.IDs = []
        self.data = dict()
        id_counter = 0
        for data_source_index in range(len(image_locations)):
            source_image_count = samples[data_source_index]
            source_data_location = data_locations[data_source_index]
            source_image_location = image_locations[data_source_index]
            source_dataset = datasets[data_source_index]

            for index in range(source_image_count):
                self.IDs.append(id_counter)
                self.data[id_counter] = [source_image_location, 
                                         source_dataset,
                                         index,
                                         np.load(os.path.join(source_data_location + self.prefix + f"{index:06d}.npy"))]
                id_counter += 1
    
    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 320, 240))
        if self.side:
            X2 = np.empty(shape=(self.batch_size, 320, 240))
        y = {
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            'bicep_out': [],
            'thigh_out': [],
            'shoulder_to_wrist_out': [],
            'leg_out': [],
            'calf_out': [],
            'wrist_out': [],
            'shoulder_to_shoulder_out': [],
        }

        start_index = index * self.batch_size + 1
        for i in range(self.batch_size):
            sample_ID = self.IDs[(start_index + i)%len(self.IDs)]
            sample_properties = self.data[sample_ID]
            current_measurement = sample_properties[3]
            current_filename = f"{self.prefix}{sample_properties[2]:06d}.png"
                
            X[i,] = cv2.imread(os.path.join(sample_properties[0]+ "images_front/" + current_filename), cv2.IMREAD_GRAYSCALE)
            if self.side:
                X2[i,] = cv2.imread(os.path.join(sample_properties[0]+ "images_side/" + current_filename), cv2.IMREAD_GRAYSCALE)
            
            height = []
                                
            if sample_properties[1] == "Surreact":
                y['chest_out'].append([current_measurement[0]])
                y['waist_out'].append([current_measurement[1]])
                y['pelvis_out'].append([current_measurement[2]])
                y['bicep_out'].append([current_measurement[4]])
                y['thigh_out'].append([current_measurement[5]])
                y['shoulder_to_wrist_out'].append([current_measurement[7]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[9]])
                y['wrist_out'].append([current_measurement[11]])
                y['shoulder_to_shoulder_out'].append([current_measurement[13]])
                if self.height:
                    height.append(current_measurement[16])
            
            if sample_properties[1] == "BodyM":
                y['chest_out'].append([current_measurement[4]])
                y['waist_out'].append([current_measurement[12]])
                y['pelvis_out'].append([current_measurement[7]])
                y['bicep_out'].append([current_measurement[2]])
                y['thigh_out'].append([current_measurement[11]])
                y['shoulder_to_wrist_out'].append([current_measurement[1]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[3]])
                y['wrist_out'].append([current_measurement[13]])
                y['shoulder_to_shoulder_out'].append([current_measurement[9]])
                if self.height:
                    height.append(current_measurement[6])
                    

        for key, value in y.items():
            y[key] = np.array(value)
            
        if self.side and self.height:
            height = np.array(height)
            return [X,X2, height], y
        if self.height:
            height = np.array(height)
            return [X, height], y
        if self.side:
            return [X,X2], y
        
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(len(self.IDs))
         

In [3]:
quickTrain = {
                'image_location': ["Export/Surreact-APose/train/", "Export/BodyM/train/"], 
                'data_location': ["Export/Surreact-APose/train/measurements/", "Export/BodyM/train/measurements/"], 
                'samples': [sample_count_surreact, sample_count_bodym], 
                'dataset': ["Surreact", "BodyM"],
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'shuffle': True}

quickValidate = {
                'image_location': ["Export/Surreact-APose/test/"],
                'data_location': ["Export/Surreact-APose/test/measurements/"],
                'samples': [validation_count],
                'batch_size': batch_size,
                'dataset': ["Surreact"],
                'seed': 69, #np.random.randint(0, 10000),
                }

train_generator = Database_Loader(**quickTrain)
validation_generator = Database_Loader(**quickValidate)

In [4]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.losses import MeanSquaredError

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        'wrist_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
    },
)

checkpoint_filepath = f'./models/{model_name}.checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[

        TensorBoard(write_graph=True, log_dir="./logs/" + folder_path),
        model_checkpoint_callback,
        stop_no_improvement
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1
)

model.save(f"./models/full/{model_name}.keras") 

2024-05-18 17:37:09.112463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 17:37:09.114270: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-18 17:37:09.114442: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 587ms/step - bicep_out_mae: 165.9365 - calf_out_mae: 84.8521 - chest_out_mae: 312.7033 - leg_out_mae: 153.2443 - loss: 882031.8750 - pelvis_out_mae: 193.1393 - shoulder_to_shoulder_out_mae: 62.8087 - shoulder_to_wrist_out_mae: 194.5066 - thigh_out_mae: 51.6678 - waist_out_mae: 253.8192 - wrist_out_mae: 67.6215 - val_bicep_out_mae: 41.9016 - val_calf_out_mae: 12.0834 - val_chest_out_mae: 44.4325 - val_leg_out_mae: 19.3484 - val_loss: 9844.0244 - val_pelvis_out_mae: 33.3624 - val_shoulder_to_shoulder_out_mae: 11.9799 - val_shoulder_to_wrist_out_mae: 52.9269 - val_thigh_out_mae: 6.8516 - val_waist_out_mae: 33.0638 - val_wrist_out_mae: 10.9951
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 540ms/step - bicep_out_mae: 27.7206 - calf_out_mae: 11.0726 - chest_out_mae: 33.3995 - leg_out_mae: 21.6696 - loss: 8095.0264 - pelvis_out_mae: 29.4845 - shoulder_to_shoulder_out_mae: 9.7322 - shoulder_to_wrist_out_mae: 35.8908 - thigh_out_mae: 5.7958 - waist_out_mae: 26.

Process Keras_worker_ForkPoolWorker-34:
Process Keras_worker_ForkPoolWorker-2:
Process Keras_worker_ForkPoolWorker-33:
Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 108, in run
    

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from keras.ops import empty


model = load_model("models/FullyTrained-Synthetic/ResNet50V2Height.keras")

#ankle	arm-length	bicep	calf	chest	forearm	height	hip	leg-length	shoulder-breadth	shoulder-to-crotch	thigh	waist	wrist
#0      1           2       3       4       5       6       7   8           9                   10                     11      12      13

def printResults(name, val1, val2):
    if val2 == "N/A":
        print(name, " - ", val1, val2)
        return 
    print(name, " - ", val1, val2, " error - ", abs(val1 - val2))



results_dataframe = pd.DataFrame(columns=["Chest", "Waist", "Pelvis", "Bicep", "Thigh", "Shoulder to wrist", "Leg", "Calf", "Wrist", "Shoulder to shoulder"])
empty = empty(shape=(1, 320, 240, 1))

for i in tqdm(range(8978)):
    image = cv2.imread(f"../bodym-dataset/Export/images_front/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
    # image2 = cv2.imread(f"../bodym-dataset/Export/images_side_flipped/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
    data = np.load(f"../bodym-dataset/Export/measurements/Avatar_{i:06d}.npy")
    height = np.load(f"../bodym-dataset/Export/measurements/height/Avatar_{i:06d}.npy", allow_pickle=True).reshape(-1,1)
    # predictions = model.predict(np.array([image]))
    predictions = model.predict([np.array([image]), height])
    # predictions = model.predict([np.array([image]),np.array([image])])
    series = pd.DataFrame({
        "Chest": abs(predictions[0][0] - data[4]), 
        "Waist": abs(predictions[1][0] - data[12]), 
        "Pelvis": abs(predictions[2][0] - data[7]), 
        "Bicep": abs(predictions[3][0] - data[2]), 
        "Thigh": abs(predictions[4][0] - data[11]), 
        "Shoulder to wrist": abs(predictions[5][0] - data[1]), 
        "Leg": abs(predictions[6][0] - data[8]),
        "Calf": abs(predictions[7][0] - data[3]), 
        "Wrist": abs(predictions[8][0] - data[13]),
        "Shoulder to shoulder": abs(predictions[9][0] - data[9])
    }, index=[0])
    results_dataframe = pd.concat([results_dataframe, series], ignore_index=True)
    # printResults("Chest", predictions[0], data[4])
    # printResults("Waist", predictions[1], data[12])
    # printResults("Pelvis", predictions[2], data[7])
    # printResults("Bicep", predictions[3], data[2])
    # printResults("Thigh", predictions[4], data[11])
    # printResults("Shoulder to wrist", predictions[5], data[1])
    # printResults("Calf", predictions[6], data[3])
    # printResults("Wrist", predictions[7], data[13])
    # printResults("Shoulder to shoulder", predictions[8], data[9])
    # printResults("Inner leg", predictions[9], data[8])

# display(results_dataframe)
results_dataframe.to_csv("BoMN+Side - real.csv", index=False)

print((results_dataframe.mean()))
print("Mean:", (results_dataframe.mean()).mean())

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
model = load_model("models/FullyTrained-Synthetic/BaselineModel100e.keras")


results_dataframe_synthetic = pd.DataFrame(columns=["Chest", "Waist", "Pelvis", "Bicep", "Thigh", "Shoulder to wrist", "Leg", "Calf", "Wrist", "Shoulder to shoulder"])


for i in tqdm(range(20000)):
    image = cv2.imread(f"./Export/Surreact-APose/test/images_front/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(f"./Export/Surreact-APose/test/images_side/Avatar_{i:06d}.png", cv2.IMREAD_GRAYSCALE)
    data = np.load(f"Export/Surreact-APose/test/measurements/Avatar_{i:06d}.npy")
    # predictions = model.predict(np.array([image]))
    predictions = model.predict([np.array([image]), np.array([image2])])
    series = pd.DataFrame({
        "Chest": abs(predictions[0][0] - data[0]),
        "Waist": abs(predictions[1][0] - data[1]),
        "Pelvis": abs(predictions[2][0] - data[2]),
        "Bicep": abs(predictions[3][0] - data[4]),
        "Thigh": abs(predictions[4][0] - data[5]),
        "Shoulder to wrist": abs(predictions[5][0] - data[7]),
        "Leg": abs(predictions[6][0] - data[8]),
        "Calf": abs(predictions[7][0] - data[9]),
        "Wrist": abs(predictions[8][0] - data[11]),
        "Shoulder to shoulder": abs(predictions[9][0] - data[13])}, index=[0])
    results_dataframe_synthetic = pd.concat([results_dataframe_synthetic, series], ignore_index=True)

# display(results_dataframe)
results_dataframe_synthetic.to_csv("SuToBoCM - synthetic.csv", index=False)

In [ ]:
print(results_dataframe_synthetic.mean())

In [ ]:
# # Averages
# import pandas
# 
# measures = realData_generator.data
# colSurreact = ["chest", "waist", "pelvis", "neck", "bicep","thigh", "knee", "arm", "leg", "calf", "head", "wrist", "armSpan", "shoulder", "torso","iLeg", "height"]
# colBodyM = ["ankle","arm-length","bicep","calf","chest","forearm","height","hip","leg-length","shoulder-breadth","shoulder-to-crotch","thigh","waist","wrist"]
# df = pandas.DataFrame(measures.values(), columns=colBodyM)
# for i in colBodyM:
#     avg = df.loc[:, i].mean()
#     print(i, df.loc[:, i].mean())

In [ ]:
# #Data modifiaction
# 
# import os
# import tqdm
# import numpy as np
# 
# avatar_count = 20000
# counter = 0
# 
# for number in tqdm.tqdm(range(avatar_count)):
#     # if number == 5:
#     #     continue
#     file = (tuple(np.load(f"../Surreact-APose/test/bodymeasurements/Avatar_{number:06d}.npy", allow_pickle=True)))
#     measurements = []
#     for value in file:
#         measurements.append(float(value * 100))
#     np.save(f"../Surreact-APose/test/measurementsCM/Avatar_{counter:06d}.npy", measurements)
#     counter += 1

In [ ]:
# Image modification
# 
# import os
# import cv2
# import numpy as np
# import tqdm
# 
# 
# avatar_count = 20000
# counter = 0
# 
# for number in range(avatar_count):
#     image = cv2.imread(f"../Surreact-APose/test/imgs_nobg_side/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     alpha_channel = image[:, :, 3]
#     body_indices = alpha_channel > 30
#     image[body_indices] = [255, 255, 255, 255]
#     indices = np.where(body_indices)
#     leftmost_pixel = np.min(indices[1])
#     rightmost_pixel = np.max(indices[1])
#     topmost_pixel = max(np.min(indices[0]), 0)
#     bottommost_pixel = min(np.max(indices[0]), 239)
#     cropped = image[topmost_pixel:bottommost_pixel, leftmost_pixel:rightmost_pixel]
# 
#     border_height = 320 - cropped.shape[0]
#     border_width = 240 - cropped.shape[1]
# 
#     # Calculate the border sizes
#     top_border = border_height // 2
#     bottom_border = border_height - top_border
#     left_border = border_width // 2
#     right_border = border_width - left_border
# 
#     # Add the border
#     bordered_image = cv2.copyMakeBorder(cropped, top_border, bottom_border, left_border, right_border, cv2.BORDER_CONSTANT, value=(0,0,0,0))
#     bordered_image = cv2.cvtColor(bordered_image, cv2.COLOR_RGBA2GRAY)
# 
# 
#     cv2.imwrite(f"./Export/Surreact-APose/test/images_side/Avatar_{counter:06d}.png", bordered_image)
#     counter +=1

In [ ]:
# import cv2
# 
# for number in range(20000):
#     image = cv2.imread(f"Export/Surreact-APose/test/images_side/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     image = cv2.flip(image, 1)
#     cv2.imwrite(f"Export/Surreact-APose/test/images_side/Avatar_{number:06d}.png", image)

In [ ]:
# Image validation

# import os
# import cv2
# import numpy as np
# from tqdm import tqdm
# 
# 
# avatar_count = 79998
# counter = 0
# 
# for number in tqdm(range(avatar_count)):
#     image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_front/Avatar_{number:06d}.png", cv2.IMREAD_UNCHANGED)
#     if image.shape != (240, 320, 4):
#         print(image.shape, number)
#     counter +=1

In [ ]:
# #FOR BODY_M DATASET
# import csv
# import numpy as np
# import os
# from collections import defaultdict
# import cv2
# import tqdm.notebook
# 
# counter = 0
# 
# os.remove("BodyM_Conversion.csv")
# for folder in tqdm.notebook.tqdm(["testA", "testB", "train"]):
# 
#     with open('../bodym-dataset/'+folder+'/subject_to_photo_map.csv', 'r') as f:
#         reader = csv.reader(f)
#         uuid_raw = list(reader)
#     
# 
#     uuid_map = defaultdict(list)
# 
#     for subject_id, photo_id in uuid_raw[1:]:
#         uuid_map[subject_id].append(photo_id)
#     image_data = {}
#     hwg_data = {}
# 
#     with open('../bodym-dataset/'+folder+'/measurements.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
# 
# 
#             for image_id in uuid_map[subject_id]:
#                 image_data[image_id] = np.array(list(map(float, line[1:])))
# 
#     with open('../bodym-dataset/'+folder+'/hwg_metadata.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
#             hwg_data[subject_id] = np.array(line[2:])
# 
# 
# 
#     
#     with open("BodyM_Conversion.csv", "a", newline="") as file:
#         writer = csv.writer(file, delimiter=";")
#         writer.writerow(["subject_id","image_id","measurements","hwg_data","avatar_id"])
#         for subject_id, images in tqdm.notebook.tqdm(uuid_map.items()):
#             for image in images:
#                 measurements = np.array(image_data[image])
#                 silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#                 silhouette = cv2.resize(silhouette,(240, 320))
#                 cv2.imwrite(os.path.join(f'../bodym-dataset/Export/images_front/Avatar_{counter:06d}.png'), silhouette)
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/Avatar_{counter:06d}.npy'), measurements)
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/height/Avatar_{counter:06d}.npy'), hwg_data[subject_id][0])
#                 np.save(os.path.join(f'../bodym-dataset/Export/measurements/weight/Avatar_{counter:06d}.npy'), hwg_data[subject_id][1])
#     
#                 silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask_left/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#                 silhouette = cv2.resize(silhouette,(240, 320))
#                 cv2.imwrite(os.path.join(f'../bodym-dataset/Export/images_side/Avatar_{counter:06d}.png'), silhouette)
#                 writer.writerow([subject_id, image, measurements, hwg_data[subject_id], counter])
#                 
#                 counter += 1
#     
#     
#         
#         
# 
# 
# 


In [ ]:
# #FOR BODY_M DATASET
# import csv
# import numpy as np
# import os
# from collections import defaultdict
# import cv2
# import tqdm.notebook
# 
# counter = 0
# for folder in tqdm.notebook.tqdm(["train"]):
# 
#     with open('../bodym-dataset/'+folder+'/subject_to_photo_map.csv', 'r') as f:
#         reader = csv.reader(f)
#         uuid_raw = list(reader)
# 
# 
#     uuid_map = defaultdict(list)
# 
#     for subject_id, photo_id in uuid_raw[1:]:
#         uuid_map[subject_id].append(photo_id)
#     image_data = {}
#     hwg_data = {}
# 
#     with open('../bodym-dataset/'+folder+'/measurements.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
# 
# 
#             for image_id in uuid_map[subject_id]:
#                 image_data[image_id] = np.array(list(map(float, line[1:])))
# 
#     with open('../bodym-dataset/'+folder+'/hwg_metadata.csv', 'r') as f:
#         reader = csv.reader(f)
# 
#         for line in reader:
#             subject_id = line[0]
#             hwg_data[subject_id] = np.array(line[2:])
# 
# 
#     for subject_id, images in tqdm.notebook.tqdm(uuid_map.items()):
#         for image in images:
#             measurements = np.append(image_data[image], float(hwg_data[subject_id][0])) 
#             measurements = np.array(measurements)
#             silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#             silhouette = cv2.resize(silhouette,(240, 320))
#             cv2.imwrite(os.path.join(f'Export/BodyM-Train/images_front/Avatar_{counter:06d}.png'), silhouette)
#             np.save(os.path.join(f'Export/BodyM-Train/measurements/Avatar_{counter:06d}.npy'), measurements)
# 
# 
#             silhouette = cv2.imread(os.path.join('../bodym-dataset/'+folder+'/mask_left/' + image + '.png'), cv2.IMREAD_GRAYSCALE)
#             silhouette = cv2.resize(silhouette,(240, 320))
#             cv2.imwrite(os.path.join(f'Export/BodyM-Train/images_side/Avatar_{counter:06d}.png'), silhouette)
#             counter += 1